<a href="https://colab.research.google.com/github/fstrike7/migracion-tecnologica-DW/blob/main/TPF_Visualizacion_Grupo7_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico Final - Visualización de Información
## Visualizando datos en un notebook.
**Grupo 7.**  
**Docente:** Diego Ariel Aizemberg  
**Integrantes:** Faustino Beatrice, Daniel Simosa y Diego Romero



## Índice
1. [Consigna 1 — Exploración de Datos (EDA)](#consigna1)
2. [Consigna 2 — Planteamiento de preguntas de investigación](#consigna2)
3. [Consigna 3 — Visualizaciones e interpretaciones](#consigna3)
4. [Conclusiones y próximos pasos](#conclusiones)
5. [Apéndice — Reproducibilidad](#apendice)


> **Contexto del dataset (resumen):**  
El trabajo se basa en el caso de **optimización del uso de red móvil y migración tecnológica en CABA**, con foco en la transición desde 2G hacia 4G/5G. Se proponen análisis por comuna/barrio, tecnología, dispositivo y tiempo, de modo de caracterizar zonas críticas, usuarios no migrados e impacto de la migración.
Se puede visitar el proyecto para la materia de DW & OLAP en este link: https://github.com/fstrike7/migracion-tecnologica-DW

> **campos esperados en el dataset:**
- `fecha` (YYYY-MM-DD), `anio`, `mes`, `nombre_mes`, `hora`
- `partido_comuna`, `barrio`, `id_celda`
- `tecnologia` (e.g., 2G/3G/4G/5G)
- `id_usuario`, `id_device`, `modelo`, `marca`, `gama`, `compatible_lte`, `compatible_5g`
- Métricas de estado: `service_status`, `release_cause1`, `release_cause2`


## Consigna 1 — Exploración de Datos (EDA) <a id="consigna1"></a>

En esta sección se carga el dataset, se realiza limpieza/preparación y un análisis exploratorio inicial:
- Descripción de distribuciones.
- Detección de valores atípicos.
- Análisis de correlaciones.
- Verificación de valores faltantes.
- Documentación de hallazgos relevantes.


In [2]:
# === Configuración e imports ===
import pandas as pd
import numpy as np
import altair as alt

import pandas as pd

BASE = "https://raw.githubusercontent.com/fstrike7/migracion-tecnologica-DW/main/data"

def read(name):
    url = f"{BASE}/{name}"
    return pd.read_csv(url, encoding="latin1")

# --- Cargar dimensiones ---
dim_celda          = read("dim_celda.csv")
dim_device         = read("dim_device.csv")
dim_fecha          = read("dim_fecha.csv")
dim_service_status = read("dim_service_status.csv")
dim_tiempo         = read("dim_tiempo.csv")
dim_usuario        = read("dim_usuario.csv")

# --- Cargar hechos ---
f_status           = read("f_status.csv")
f_registraciones   = read("f_registraciones.csv")

# vista de info basica
print("f_status.shape ->", f_status.shape)
display(f_status.head())
display(f_status.tail())


f_status.shape -> (200000, 8)


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda
0,1,20250516,141,2025-05-16 00:01:41,3G,24,21788,WCN275X
1,2,20250516,142,2025-05-16 00:01:42,3G,17,79404,WBM080X
2,3,20250516,142,2025-05-16 00:01:42,3G,24,42913,WBO003Z
3,4,20250516,143,2025-05-16 00:01:43,2G,17,70339,CBC070O
4,5,20250516,148,2025-05-16 00:01:48,3G,24,100596,WBA018Z


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda
199995,199996,20250516,2753,2025-05-16 00:27:53,2G,21,2884,VCN454Y
199996,199997,20250516,2754,2025-05-16 00:27:54,2G,21,2974,CUE049C
199997,199998,20250516,2755,2025-05-16 00:27:55,2G,21,2972,CFN003O
199998,199999,20250516,2755,2025-05-16 00:27:55,2G,21,2988,GES021C
199999,200000,20250516,2758,2025-05-16 00:27:58,2G,21,2986,CDE030B


In [3]:

# === Construcción de df_main a partir de hechos + dimensiones ===
import pandas as pd

def _has_cols(df, cols):
    return all(c in df.columns for c in cols)

df_main = f_status.copy()

# Join con dimensiones si existen las keys
if 'id_celda' in df_main.columns and 'id_celda' in dim_celda.columns:
    df_main = df_main.merge(dim_celda, on='id_celda', how='left')
if 'id_device' in df_main.columns and 'id_device' in dim_device.columns:
    df_main = df_main.merge(dim_device, on='id_device', how='left')
if 'id_service_status' in df_main.columns and 'id_service_status' in dim_service_status.columns:
    df_main = df_main.merge(dim_service_status, on='id_service_status', how='left')
if 'id_fecha' in df_main.columns and 'id_fecha' in dim_fecha.columns:
    df_main = df_main.merge(dim_fecha, on='id_fecha', how='left')
if 'id_tiempo' in df_main.columns and 'id_tiempo' in dim_tiempo.columns:
    df_main = df_main.merge(dim_tiempo, on='id_tiempo', how='left')

# Parse de fecha si existe
for col in ('fecha', 'Fecha', 'FECHA'):
    if col in df_main.columns:
        df_main['fecha'] = pd.to_datetime(df_main[col], errors='coerce')
        break

# Normalización de columnas "estándar" esperadas por el notebook
def alias_col(df, candidates, new_name):
    for c in candidates:
        if c in df.columns:
            if new_name not in df.columns:
                df[new_name] = df[c]
            return
# partido_comuna puede venir como 'partido_comuna' / 'comuna' / 'barrio' / 'localidad'
alias_col(df_main, ['partido_comuna','comuna','barrio','localidad'], 'partido_comuna')
# tecnologia podría existir en f_status o alguna dim; si existe, lo dejamos como está
# service_status puede ser 'service_status' o 'status' o 'estado'
alias_col(df_main, ['service_status','status','estado'], 'service_status')
# hora puede venir de dim_tiempo ('hora','hour','hh')
alias_col(df_main, ['hora','hour','hh'], 'hora')
# compat flags si existen
for src, dst in [(['compatible_lte','lte_compatible','is_lte'], 'compatible_lte'),
                 (['compatible_5g','is_5g','5g_compatible'], 'compatible_5g')]:
    alias_col(df_main, src, dst)

print("df_main.shape ->", df_main.shape)
display(df_main.head(5))


df_main.shape -> (200000, 40)


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda,sitio,tecnologia,...,semana_nro,dia_semana_nro,dia_semana,dia_semana_corto,trimestre,año_trimestre,tiempo,hora,minuto,segundo
0,1,20250516,141,2025-05-16 00:01:41,3G,24,21788,WCN275X,CN275,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:41,0,1,41
1,2,20250516,142,2025-05-16 00:01:42,3G,17,79404,WBM080X,BM080,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:42,0,1,42
2,3,20250516,142,2025-05-16 00:01:42,3G,24,42913,WBO003Z,BO003,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:42,0,1,42
3,4,20250516,143,2025-05-16 00:01:43,2G,17,70339,CBC070O,BC070,2G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:43,0,1,43
4,5,20250516,148,2025-05-16 00:01:48,3G,24,100596,WBA018Z,BA018,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:48,0,1,48


In [7]:
# === Valores faltantes ===
if not f_status.empty:
    missing = f_status.isna().sum().sort_values(ascending=False)
    missing = missing[missing > 0]
    display(missing)
else:
    print("El DataFrame está vacío (placeholder). Cargue el dataset en DATA_PATH.")

,0


In [8]:
# === Duplicados ===
if not df_main.empty:
    print("Registros duplicados:", df_main.duplicated().sum())
else:
    print("El DataFrame está vacío (placeholder).")

Registros duplicados: 0


In [9]:
# === Distribuciones (describe) ===
if not df_main.empty:
    display(df_main.describe(include='all'))
else:
    print("El DataFrame está vacío (placeholder).")

,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda,sitio,tecnologia,...,semana_nro,dia_semana_nro,dia_semana,dia_semana_corto,trimestre,año_trimestre,tiempo,hora,minuto,segundo
count,200000.000000,200000.0,200000.00000,200000,200000,200000.000000,200000.000000,200000,200000,200000,...,200000.0,200000.0,200000,200000,200000,200000,200000,200000.0,200000.000000,200000.000000
unique,NaN,NaN,NaN,1006,2,NaN,NaN,20323,5184,3,...,NaN,NaN,1,1,1,1,1006,NaN,NaN,NaN
top,NaN,NaN,NaN,2025-05-16 00:16:22,3G,NaN,NaN,WBH057Z,BH057,3G,...,NaN,NaN,viernes,vier,Q2,2025/Q2,00:16:22,NaN,NaN,NaN
freq,NaN,NaN,NaN,509,178839,NaN,NaN,353,1281,175282,...,NaN,NaN,200000,200000,200000,200000,509,NaN,NaN,NaN
mean,100000.500000,20250516.0,1374.33085,NaN,NaN,24.507130,64984.188490,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,13.445535,29.777350
min,1.000000,20250516.0,141.00000,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,1.000000,0.000000
25%,50000.750000,20250516.0,1011.00000,NaN,NaN,24.000000,36198.750000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,10.000000,14.000000
50%,100000.500000,20250516.0,1308.00000,NaN,NaN,24.000000,68131.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,13.000000,30.000000
75%,150000.250000,20250516.0,1650.00000,NaN,NaN,24.000000,97363.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,16.000000,45.000000
max,200000.000000,20250516.0,2758.00000,NaN,NaN,37.000000,116765.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,27.000000,59.000000


In [10]:
# === Correlaciones (variables numéricas) ===
if not df_main.empty:
    num_cols = df_main.select_dtypes(include=np.number).columns
    if len(num_cols) >= 2:
        corr = df_main[num_cols].corr(numeric_only=True)
        corr_reset = corr.reset_index().melt('index')
        corr_reset.columns = ['var_x', 'var_y', 'correlacion']
        chart = alt.Chart(corr_reset).mark_rect().encode(
            x=alt.X('var_x:O', sort=None, title='Variable X'),
            y=alt.Y('var_y:O', sort=None, title='Variable Y'),
            tooltip=['var_x', 'var_y', alt.Tooltip('correlacion:Q', format='.2f')]
        ).properties(title='Matriz de correlación (numéricas)')
        display(chart)
    else:
        print("No hay suficientes variables numéricas para calcular correlaciones.")
else:
    print("El DataFrame está vacío (placeholder).")

alt.Chart(...)

In [11]:
# === Detección simple de outliers (z-score) ===
if not df_main.empty:
    from scipy.stats import zscore
    num_cols = df_main.select_dtypes(include=np.number).columns
    if len(num_cols) > 0:
        zscores = df_main[num_cols].apply(zscore).abs()
        outlier_counts = (zscores > 3).sum().sort_values(ascending=False)
        display(outlier_counts[outlier_counts > 0])
    else:
        print("No hay columnas numéricas para evaluar outliers.")
else:
    print("El DataFrame está vacío (placeholder).")

/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  results[i] = self.func(v, *self.args, **self.kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/apply.py:1081: RuntimeWarning: Precision loss occurred in moment calc

,0
id_service_status,9906
comuna,9442


**Hallazgos EDA (completar):**
- [ ] *Ejemplo:* Falta de `compatible_lte` en X% de registros → revisar fuente/criterio de imputación.
- [ ] *Ejemplo:* Distribución de `tecnologia` está sesgada hacia 2G en comunas A, B.
- [ ] *Ejemplo:* Correlaciones relevantes entre `gama` y uso de `tecnologia` moderna.


## Consigna 2 — Planteamiento de preguntas de investigación <a id="consigna2"></a>

A continuación, se formulan preguntas significativas a responder con el dataset seleccionado. Estas preguntas se **basan en el trabajo de DW & OLAP** del grupo (optimización del uso de red móvil y migración tecnológica en CABA).


**Preguntas (adaptadas del trabajo DW & OLAP):**
1. **Identificación de zonas críticas**  
   - ¿Qué porcentaje de conexiones por comuna/barrio se realiza aún en redes 2G?
   - ¿Cómo se distribuyen las antenas por tipo de red (2G, 4G, 5G) y proveedor en CABA?
2. **Caracterización de usuarios no migrados**  
   - ¿Qué porcentaje de dispositivos conectados a 2G no son compatibles con LTE (4G) o 5G por comuna?
3. **Impacto de la migración**  
   - De los clientes que pasan a 4G/5G, ¿cuántos no vuelven a conectarse a 2G en el período posterior?  
   - ¿Cómo evolucionó la tasa de fallas de conexión de las celdas antes y después de la migración?  
   - ¿En qué franjas horarias se registra mayor saturación de red en zonas con coexistencia 2G/4G?


## Consigna 3 — Visualizaciones e interpretaciones <a id="consigna3"></a>

Se construyen visualizaciones para responder a las preguntas, siguiendo principios de buenas prácticas vistos en clase. Cada visual se acompaña de una **breve interpretación**.


In [12]:
# === P1: Porcentaje de conexiones por tecnología y comuna/mes ===
if not df_main.empty:
    required_cols = {'anio', 'nombre_mes', 'partido_comuna', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        grouped = (
            df_main[df_main['tecnologia'].isin(['2G','3G','4G','5G'])]
            .groupby(['anio','nombre_mes','partido_comuna','tecnologia'], as_index=False)
            .size()
            .rename(columns={'size':'total'})
        )
        # Calcular porcentaje dentro de cada (anio, nombre_mes, partido_comuna)
        grouped['porcentaje'] = grouped.groupby(['anio','nombre_mes','partido_comuna'])['total']                                        .transform(lambda x: (x / x.sum()) * 100)

        chart = alt.Chart(grouped).mark_bar().encode(
            x=alt.X('tecnologia:N', title='Tecnología'),
            y=alt.Y('porcentaje:Q', title='% de conexiones'),
            color='tecnologia:N',
            column=alt.Column('partido_comuna:N', title='Comuna/Barrio'),
            tooltip=['anio','nombre_mes','partido_comuna','tecnologia', alt.Tooltip('porcentaje:Q', format='.2f')]
        ).properties(title='% de conexiones por tecnología y comuna')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


Faltan columnas para esta visualización: {'anio'}


**Interpretación (completar):**  
- [ ] Comunas con mayor dependencia de 2G: …  
- [ ] Comunas con mayor adopción 4G/5G: …  
- [ ] Estacionalidad/variación mensual notable: …  


In [13]:
# === P2: % de dispositivos no compatibles con LTE/5G por comuna ===
if not df_main.empty:
    # Se asume la existencia de flags booleanos (o 0/1) `compatible_lte`, `compatible_5g`
    required_cols = {'partido_comuna', 'compatible_lte', 'compatible_5g', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        # No compatible si (lte == False) y (5g == False)
        df_main['no_compatible_moderno'] = (~df_main['compatible_lte'].astype(bool)) & (~df_main['compatible_5g'].astype(bool))

        by_comuna = df_main.groupby('partido_comuna', as_index=False).agg(
            total=('no_compatible_moderno', 'size'),
            no_compatibles=('no_compatible_moderno', 'sum')
        )
        by_comuna['porcentaje_no_compat'] = (by_comuna['no_compatibles'] / by_comuna['total']) * 100

        chart = alt.Chart(by_comuna).mark_bar().encode(
            x=alt.X('partido_comuna:N', sort='-y', title='Comuna/Barrio'),
            y=alt.Y('porcentaje_no_compat:Q', title='% no compatibles (LTE/5G)'),
            tooltip=['partido_comuna', alt.Tooltip('porcentaje_no_compat:Q', format='.2f')]
        ).properties(title='% de dispositivos no compatibles por comuna')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


Faltan columnas para esta visualización: {'compatible_lte', 'compatible_5g'}


**Interpretación (completar):**  
- [ ] Comunas con mayor porcentaje de dispositivos no compatibles: …  
- [ ] Posibles causas (gama, ingresos, antigüedad de dispositivos): …  
- [ ] Implicancias para políticas de migración (beneficios/planes): …  


In [14]:
# === P3a: Clientes que migran a 4G/5G y no regresan a 2G ===
if not df_main.empty:
    required_cols = {'id_usuario', 'fecha', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        df_main['fecha'] = pd.to_datetime(df_main['fecha'], errors='coerce')
        # Para cada usuario, detectar primera conexión 4G/5G y si luego hubo regresos a 2G
        tech_map = {'2G':1, '3G':2, '4G':3, '5G':4}
        df_main['tech_order'] = df_main['tecnologia'].map(tech_map).fillna(0)

        # Primera fecha con 4G/5G
        modern = df_main[df_main['tecnologia'].isin(['4G','5G'])]
        first_modern = modern.sort_values('fecha').groupby('id_usuario', as_index=False).first()[['id_usuario','fecha']]
        first_modern.columns = ['id_usuario','fecha_first_modern']

        merged = df_main.merge(first_modern, on='id_usuario', how='left')
        # Usuarios que tienen 4G/5G al menos una vez
        has_modern = merged[~merged['fecha_first_modern'].isna()]['id_usuario'].unique()
        # Entre esos, verificar si luego de esa fecha hay usos de 2G
        after = merged[merged['fecha'] > merged['fecha_first_modern']]
        returned_to_2g = after[after['tecnologia'] == '2G']['id_usuario'].unique()

        total_migrated = len(has_modern)
        no_return = total_migrated - len(np.intersect1d(has_modern, returned_to_2g))

        print("Usuarios con uso 4G/5G (al menos una vez):", total_migrated)
        print("De ellos, NO regresaron a 2G posteriormente:", no_return)
        if total_migrated > 0:
            print("Porcentaje:", round((no_return/total_migrated)*100, 2), "%")
    else:
        print("Faltan columnas para este análisis:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


Usuarios con uso 4G/5G (al menos una vez): 2075
De ellos, NO regresaron a 2G posteriormente: 2075
Porcentaje: 100.0 %


In [15]:
# === P3b: Evolución de fallas antes/después de migración ===
if not df_main.empty:
    required_cols = {'fecha', 'partido_comuna', 'service_status'}
    if required_cols.issubset(df_main.columns):
        df_main['fecha'] = pd.to_datetime(df_main['fecha'], errors='coerce')
        # Ejemplo: considerar 'FAIL' como evento de falla
        df_main['falla'] = (df_main['service_status'].astype(str).str.upper() == 'FAIL').astype(int)
        daily = df_main.groupby('fecha', as_index=False)['falla'].mean()
        chart = alt.Chart(daily).mark_line(point=True).encode(
            x=alt.X('fecha:T', title='Fecha'),
            y=alt.Y('falla:Q', title='Tasa de fallas (promedio diario)'),
            tooltip=[alt.Tooltip('fecha:T'), alt.Tooltip('falla:Q', format='.3f')]
        ).properties(title='Evolución de tasa de fallas (diario)')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


alt.Chart(...)

In [16]:
# === P3c: Saturación por franja horaria en zonas con 2G y 4G ===
if not df_main.empty:
    required_cols = {'fecha', 'hora', 'id_celda', 'partido_comuna', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        subset = df_main[df_main['tecnologia'].isin(['2G','4G'])].copy()
        by_hour = subset.groupby(['hora','partido_comuna'], as_index=False).size().rename(columns={'size':'conexiones'})
        chart = alt.Chart(by_hour).mark_line(point=True).encode(
            x=alt.X('hora:O', title='Hora del día'),
            y=alt.Y('conexiones:Q', title='Conexiones'),
            color='partido_comuna:N',
            tooltip=['partido_comuna','hora','conexiones']
        ).properties(title='Conexiones por hora en zonas con 2G/4G')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


alt.Chart(...)

**Interpretaciones (completar):**
- [ ] **P1:** …
- [ ] **P2:** …
- [ ] **P3:** …


## Conclusiones y próximos pasos <a id="conclusiones"></a>
- **Conclusión principal:** *(resumir los hallazgos que responden las preguntas)*
- **Limitaciones:** *(calidad/volumen de datos, supuestos, variables no disponibles)*
- **Próximos pasos:** *(profundizar con más variables, segmentaciones, pruebas A/B, dashboards, etc.)*


## Para ejecutar en local <a id="apendice"></a>
- **Versiones sugeridas:**
  - Python >= 3.10
  - pandas, numpy, altair, scipy
- **Estructura esperada:**
  ```
  /data
    └── dataset_movil_caba.csv  # dataset real usado
  notebook.ipynb
  ```
- **Cómo ejecutar localmente:**
  1. Crear entorno: `python -m venv .venv && source .venv/bin/activate` (Linux/Mac) o `.venv\Scripts\activate` (Windows).
  2. Instalar dependencias: `pip install pandas numpy altair scipy`.
  3. Colocar el dataset en `data/` y actualizar `DATA_PATH` en la celda de carga.
  4. Ejecutar todas las celdas en orden.
